In [10]:
import altair as alt
import pandas as pd
import panel as pn
#pn.extension('vega')
#pn.extension('tabulator')

In [11]:
#read the file
file = r'Clastic_TS_Altair6.xlsx'
source6 = pd.read_excel(file,index_col=False)


In [12]:
source6.head()

,Sample,Porosity,Permeability,sqrt(k/phi),lPerm,image
0,1,0.292,1770.0,77.86,3.25,./data/1.png
1,2,0.310,2200.0,84.24,3.34,./data/Great_SS.png
2,3,0.218,28.4,11.41,1.45,./data/12.png
3,4,0.144,23.0,12.64,1.36,./data/MediumSS.png
4,5,0.159,4.9,5.55,0.69,./data/DistalTurbidite.png


In [13]:
# Brush for selection
brush = alt.selection(type='interval')



chart = alt.Chart(source6).mark_circle(size=400).encode(
    alt.X('Porosity:Q',
        scale=alt.Scale(domain=(0, 0.4))),
    #y='Perm',
    alt.Y('Permeability:Q',scale=alt.Scale(type='log', base=10,domain=(.0001, 10000))),
    #color='LPerm:Q',
    color = alt.Color('sqrt(k/phi):Q', scale=alt.
                      Scale(scheme = 'rainbow'),legend=None),
    #alt.Y('LPerm:Q',scale=alt.Scale(type='log', base=10,domain=(.0001, 10000))),
    tooltip=['image','Sample','Porosity','Permeability','sqrt(k/phi)'],  # Must be a list for the image to render
    
).properties(
    width=500,
    height=500, 

    title='Porosity vs. Permeability Cross Plot: hover over Sample to see Thin Section',

).add_selection(brush)
    
vega_pane = pn.pane.Vega(chart.add_selection(
    brush
))
    

    


# Base chart for data tables
ranked_text = alt.Chart(source6).mark_text(align='right').encode(
    y=alt.Y('row_number:O',axis=None)
).transform_window(
    row_number='row_number()'
).transform_filter(
    brush
).transform_window(
    rank='rank(row_number)'
).transform_filter(
    alt.datum.rank<16
)


# Data Tables
Porosity = ranked_text.encode(text='Porosity:N').properties(title=alt.TitleParams(text='Porosity', align='right'))
Permeability = ranked_text.encode(text='Permeability:N').properties(title=alt.TitleParams(text='Permeability', align='right'))
Sample = ranked_text.encode(text='Sample:N').properties(title=alt.TitleParams(text='Sample', align='right'))
Image = ranked_text.encode(text='image:N').properties(title=alt.TitleParams(text ='image', align='right'))
sqrt = ranked_text.encode(text='sqrt(k/phi):N').properties(title=alt.TitleParams(text ='sqrt(k/phi)', align='right'))


text = alt.hconcat(Sample, Porosity, Permeability,sqrt) # Combine data tables

# Build chart
alt.hconcat(
    chart,
    text
).resolve_legend(
    color="independent"
).configure_view(strokeWidth=0)




alt.HConcatChart(...)